### Data importing

In [1]:
import zipfile
import os

# Path to the uploaded .zip file
zip_file_path = '/content/drive/MyDrive/data.zip'

# Directory where you want to extract the files
extract_dir = '/content/data_set'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete!")


Extraction complete!


#### Importing required Libraries

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3, MobileNetV2, DenseNet121, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


#### Data Preprocessing

In [3]:
data_dir = '/content/data_set/data'

Data Augmentaion for training and testing data

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting data into training and validation
)

In [5]:
# Data preparation for validation and test sets
test_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

In [6]:
# Train generator
print("Image for training dataset")
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
print()
# Test generator
print("Images for validation dataset")
validation_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Image for training dataset
Found 3200 images belonging to 10 classes.

Images for validation dataset
Found 800 images belonging to 10 classes.


#### Model Building and training

A function which takes different base models and train them.

In [7]:
def create_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)  # predictions are 10 because there are 10 classes of data
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False  # ignoring training for the base model layers

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


List of base models used for this project

In [8]:
base_models = [
    ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
]

16705208/16705208 [==============================] - 0s 0us/step


In [9]:
# Initialize variables to store best model and accuracy
best_model = None
best_accuracy = 0.0

Iterating through each model and traing it and updating the variable best_model when a best accuracy comes

In [ ]:
for base_model in base_models:
    model = create_model(base_model)
    model_name = base_model.name

    # Define callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(f'{model_name}.h5', monitor='val_loss', save_best_only=True)
    ]
    print(f'Training model: {model_name}')
    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=callbacks
    )

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(validation_generator)

    # Update the best model
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model_name

print(f'Best model: {best_model} with accuracy: {best_accuracy}')

Training model: resnet50
Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 2.3235 - accuracy: 0.1541

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 112s 1s/step - loss: 2.3235 - accuracy: 0.1541 - val_loss: 2.2190 - val_accuracy: 0.1238
Epoch 2/10
100/100 [==============================] - 99s 990ms/step - loss: 2.1487 - accuracy: 0.2244 - val_loss: 2.1134 - val_accuracy: 0.2025
Epoch 3/10
100/100 [==============================] - 102s 1s/step - loss: 2.0871 - accuracy: 0.2387 - val_loss: 2.0661 - val_accuracy: 0.2325
Epoch 4/10
100/100 [==============================] - 104s 1s/step - loss: 2.0233 - accuracy: 0.2625 - val_loss: 2.0970 - val_accuracy: 0.2738
Epoch 5/10
100/100 [==============================] - 101s 1s/step - loss: 2.0051 - accuracy: 0.2784 - val_loss: 1.9977 - val_accuracy: 0.2525
Epoch 6/10
100/100 [==============================] - 100s 997ms/step - loss: 1.9655 - accuracy: 0.2891 - val_loss: 1.9852 - val_accuracy: 0.2812
Epoch 7/10
100/100 [==============================] - 101s 1s/step - loss: 1.9670 - accuracy: 0.2869 - val_loss: 1.9906 - val_accuracy: 0.2812
Epoch

In [ ]:
# Retrieve class indices mapping
class_indices = train_generator.class_indices
# Reverse the mapping to get a dictionary that maps indices to class names
index_to_class = {v: k for k, v in class_indices.items()}

In [ ]:
index_to_class

loading the best model

In [ ]:
def load_model(model_name):
    base_model_map = {
        'resnet50': ResNet50,
        'vgg16': VGG16,
        'inception_v3': InceptionV3,
        'mobilenet_v2': MobileNetV2,
        'densenet121': DenseNet121,
        'efficientnetb0': EfficientNetB0
    }
    base_model = base_model_map[model_name](weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = create_model(base_model)
    model.load_weights(f'{model_name}.h5')
    return model

best_model = load_model(best_model)

#### Prediction

In [ ]:
def predict_image(model, img_path, index_to_class):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_class_name = index_to_class[predicted_index]
    return predicted_class_name, prediction

# Example usage
img_path = '/content/data_set/data/bacterial_leaf_blight/bacterial_leaf_blight1015.jpg'
predicted_class_name, probabilities = predict_image(best_model, img_path, index_to_class)
print(f'Predicted class: {predicted_class_name} with probabilities: {probabilities}')